In [1]:
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, EstimatorV2 as Estimator

from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt


In [2]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

# Load the dataset
data = load_breast_cancer()

# Convert to a pandas DataFrame for easier manipulation
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Display the first few rows
print(df.head())


   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \
0             

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap


In [4]:
# Load dataset from sklearn
data = load_breast_cancer()
X, y = data.data, data.target

# Use only first 2 features for quick demo (quantum circuits get heavy with >3 features)
X = X[:, :2]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
# 2D feature map (since we use 2 features)
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)


In [ ]:
# Quantum kernel using fidelity
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

# Quantum SVM classifier
qsvc = QSVC(quantum_kernel=quantum_kernel)
X_small = X_train[:30]
y_small = y_train[:30]
qsvc.fit(X_small, y_small)

# Train
qsvc.fit(X_train, y_train)

# Test accuracy
score = qsvc.score(X_test, y_test)
print("QSVC Accuracy:", score)


In [ ]:
import numpy as np

# Create meshgrid
xx, yy = np.meshgrid(np.linspace(X[:,0].min(), X[:,0].max(), 50),
                     np.linspace(X[:,1].min(), X[:,1].max(), 50))

grid = np.c_[xx.ravel(), yy.ravel()]
grid = scaler.transform(grid)

# Predict
Z = qsvc.predict(grid)
Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, edgecolors='k')
plt.title("Quantum SVM Decision Boundary")
plt.show()
